In [2]:
import nibabel as nib
import numpy as np
import os
import pandas as pd 
import glob
from sklearn.linear_model import LinearRegression
from scipy import stats

In [3]:
STUDY_PATH="/NAS/deathrow/protocoles/predistim/"
#PATH_SCRIPT=DIR+"code/"
#LOG_DIR=PATH_SCRIPT+"log/" #pour les sorties de qbatch
#VB_DATA_PATH=DIR+"volbrain/"
#T1_DATA_PATH=DIR+"2018-07-30_t1_segmentations/"
#MULTIGRE_DATA_PATH=DIR+"2018-12-12_3dmultigre/"
#FS_PATH=DIR+"FS60_VB/"
#PATH_SCREENSHOT=DIR+"screenshots/"
#MNI152_PATH="/home/global/fsl5011/data/standard/MNI152_T1_1mm.nii.gz"

In [26]:
SUBJ='01011BB'
img = nib.load(os.path.join(STUDY_PATH,'HCP',SUBJ,'T1w','fsaverage_LR32k',SUBJ+'.L.midthickness.32k_fs_LR.surf.gii'))
img_data = [x.data for x in img.darrays]
print(img_data)
SUBJ='01011BB'
img = nib.load(os.path.join(STUDY_PATH,'HCP',SUBJ,'MNINonLinear','fsaverage_LR32k',SUBJ+'.L.midthickness.32k_fs_LR.surf.gii'))
img_data = [x.data for x in img.darrays]
print(img_data)

[array([[ -2.9772344, -45.495827 ,  35.745655 ],
       [-15.519616 , -37.501694 ,  61.345802 ],
       [-43.98158  ,   2.560153 ,  34.737312 ],
       ...,
       [-56.1724   , -43.956932 ,  -4.6327863],
       [-57.408497 , -45.052586 ,  -3.4137893],
       [-56.099327 , -45.96313  ,  -4.446026 ]], dtype=float32), array([[   68,    12,     0],
       [  180,    12,    68],
       [  180,    13,    12],
       ...,
       [ 8440, 32491, 21432],
       [ 8440,  8441, 32491],
       [    9,  8440, 21432]], dtype=int32)]
[array([[ -3.3621507, -48.1586   ,  31.341684 ],
       [-19.735895 , -43.605885 ,  68.77835  ],
       [-50.71095  ,  -4.395582 ,  45.21349  ],
       ...,
       [-62.377625 , -51.461212 ,  -3.1860018],
       [-63.8995   , -52.591827 ,  -1.9561766],
       [-62.14521  , -53.436584 ,  -3.97922  ]], dtype=float32), array([[   68,    12,     0],
       [  180,    12,    68],
       [  180,    13,    12],
       ...,
       [ 8440, 32491, 21432],
       [ 8440,  8441, 324

In [23]:
SUBJ='01011BB'
img = nib.load(os.path.join(STUDY_PATH,'HCP',SUBJ,'MNINonLinear','fsaverage_LR32k',SUBJ+'.L.midthickness.32k_fs_LR_va.shape.gii'))
img_data = [x.data for x in img.darrays]
print(img_data)
SUBJ='01026AF'
img = nib.load(os.path.join(STUDY_PATH,'HCP',SUBJ,'MNINonLinear','fsaverage_LR32k',SUBJ+'.L.midthickness.32k_fs_LR_va.shape.gii'))
img_data = [x.data for x in img.darrays]
print(img_data)

32492


Correlation (variable explicative)
    Dopa 
    
Regression (variable confondante) 
    Age 
    Durée de la maladie 
    Poids 
    Sexe

In [67]:
img = nib.load(os.path.join(STUDY_PATH,'SurfaceMapping','QSM.L.32k_fs_LR.shape.gii'))
img_data = [x.data for x in img.darrays]
cur_data = np.reshape(img_data,(len(img_data[0]),len(img_data)))

print(type(cur_data))
print(cur_data.shape)

<class 'numpy.ndarray'>
(32492, 1)


In [74]:
ClinicalData = pd.read_excel(os.path.join(STUDY_PATH,'Data_Cli_2020_06_22.xlsx'))
print(ClinicalData.describe)

print(ClinicalData[ClinicalData.PatientID ==  '01007HC'].DOPA)
ClinicalData['Poids']=pd.to_numeric(ClinicalData['Poids'])
ClinicalData['Age']=pd.to_numeric(ClinicalData['Age'])
ClinicalData['Duree']=pd.to_numeric(ClinicalData['Duree'])
ClinicalData['DOPA']=pd.to_numeric(ClinicalData['DOPA'])


<bound method NDFrame.describe of     PatientID  Age  Sexe  Duree  Poids  DOPA
0     01001SR   67   1.0   15.0   76.0  51.9
1     01002TM   59   1.0    8.0   88.0  97.5
2     01003SJ   61   1.0   13.0   89.0  71.4
3     01004DJ   65   2.0   16.0   59.0  90.7
4     01005LB   65   1.0   17.0   81.0  85.4
..        ...  ...   ...    ...    ...   ...
829   20025DA   61   2.0   11.0   74.0  73.1
830   20026ME   59   1.0    6.0   98.0  96.3
831   20027RP   64   1.0    7.0   78.0  82.4
832   20028DN   69   2.0    8.0   85.0  87.5
833   20029DA   70   2.0   13.0   48.0  93.8

[834 rows x 6 columns]>
6    78.7
Name: DOPA, dtype: float64


In [ ]:
dir_names=glob.glob(os.path.join(STUDY_PATH,'SurfaceMapping','*','QSM.R.32k_fs_LR.shape.gii'))
dopa_group=[]
QSM_group=[]
for cpt,dir_name in enumerate(dir_names) :
    print(cpt)
    SUBJID=os.path.split(os.path.split(dir_name)[0])[1]
    print(SUBJID)
    ind=ClinicalData[ClinicalData.PatientID ==  SUBJID].index.values.astype(int)
    ima_path=os.path.join(STUDY_PATH,'SurfaceMapping',SUBJID,'QSM.R.32k_fs_LR.shape.gii')
    if (np.shape(ind)[0]>0) & os.path.exists(ima_path) :
        print(ClinicalData.loc[int(ind)].values)
        #age=ClinicalData.loc[int(ind)].at['Age'].astype(int) 
        #dur=ClinicalData.loc[int(ind)].at['Duree'].astype(int) 
        #poids=ClinicalData.loc[int(ind)].at['Poids'].astype(int) 
        #sexe=ClinicalData.loc[int(ind)].at['Sexe'].astype(int)        
        if ClinicalData.loc[int(ind)].at['DOPA'].astype(int) > 0 :
            dopa_group.append(ClinicalData.loc[int(ind)].at['DOPA'].astype(int))  
            img = nib.load(ima_path)
            img_data = [x.data for x in img.darrays]
            QSM_group.append(np.reshape(img_data,(len(img_data[0]),len(img_data))))

QSM_values=np.asarray(QSM_group)
dopa_values=np.asarray(dopa_group)

In [115]:
p_val_group=[]
r_val_group=[]
for cpt in range(32492):
    r_val,p_val=stats.pearsonr(QSM_values[:,cpt,0], dopa_values)
    p_val_group.append(p_val)
    r_val_group.append(r_val)

In [160]:
image = GiftiImage()
image.version = "version_string"
#image.darray = POINTER(POINTER(GiftiDataArray))()
image.data = None
#image.numDA=0
image.swapped = 0
image.compressed = 0

AttributeError: can't set attribute

In [162]:
# type(im) = GiftiImage
# type(ver) = str
def set_version(im, ver):
    image.version = c_char_p(ver)

# type(im) = GiftiImage
# type(numDA) = int
def set_numDA(im, numDA):
    im.numDA = c_int(numDA)

# type(im) = GiftiImage
# type(md) = GiftiMetaData
def set_meta_data(im, md):
    im.meta = md

# type(im) = GiftiImage
# type(da) = GiftiDataArray
def add_data_array(im, da):
    cur_numda = im.numDA

    # Create a pointer to the new dataarray
    da_ptr = pointer(da.data)
    # Grab the pointer to the image's dataarrays
    ptr = image.darray

    # Create a new dataarray array pointer
    ar = (POINTER(GiftiDataArray)*(cur_numda+1))()
    # We need to cast the resulting pointer for use by C
    ar = cast(ar, POINTER(POINTER(GiftiDataArray)))

    # Copy all of the current da's to the new array.  This just copies the pointers!
    for i in xrange(num_da):
        ar[i] = im.darray[i]

    # Add the new data array to the image's data
    ar[num_da] = da_ptr

    # Reassign the pointer
    im.darray = ar

    # Tell the image it has an extra DA now
    cur_numda += 1
    im.numDA = c_int(cur_numda)

# type(da) = GiftiDataArray
# type(axis) = int
# type(val) = int
def set_da_dim(da, axis, val):
    # Simple setter.  However, the axis variable here is a
    # python array index (as the da dims is an array)
    # To properly assign the value, the val variable must
    # be usable by C, so we must form a c_int type.
    # This is true for all Setters.
    da.dims[axis] = c_int(val)

# type(filename) = string
# type(im) = GiftiImage
def write_image(filename, im):
    return(gifti_write_image, im, filename, 1)

In [140]:
from nibabel.gifti import GiftiImage, GiftiDataArray
from nibabel import gifti
another_img = nib.GiftiImage(img.header)
intent = 'NIFTI_INTENT_POINTSET'
another_img.add_gifti_data_array(GiftiDataArray(r_val_group,intent))
gifti.write(another_img, os.path.join(STUDY_PATH,'SurfaceMapping','QSM.DOPA.r_val.R.32k_fs_LR.shape.gii'))

/home/global/anaconda37/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: giftiio.write function deprecated. Use nibabel.load() instead.

* deprecated from version: 2.1
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 4.0
  


In [141]:
nib.save(another_img, os.path.join(STUDY_PATH,'SurfaceMapping','QSM_2.DOPA.r_val.R.32k_fs_LR.shape.gii'))

In [142]:
another_img.to_filename(os.path.join(STUDY_PATH,'SurfaceMapping','QSM_3.DOPA.r_val.R.32k_fs_LR.shape.gii'))

In [164]:
write_image(os.path.join(STUDY_PATH,'SurfaceMapping','QSM_4.DOPA.r_val.R.32k_fs_LR.shape.gii'),another_img)

NameError: name 'gifti_write_image' is not defined

In [100]:
ima_path=os.path.join(STUDY_PATH,'SurfaceMapping','QSM.DOPA.r_val.R.32k_fs_LR.shape.gii')
img = nib.load(ima_path)
img_data = [x.data for x in img.darrays]
QSM_group.append(np.reshape(img_data,(len(img_data[0]),len(img_data))))

(95, 32492, 1) (95,)
